In [ ]:
import pandas as pd
import logging
import os


log_path = r"C:\Users\Rasulbek907\Desktop\Hotel Booking Cancellation Prediction\Log\data_loader.log"
os.makedirs(os.path.dirname(log_path), exist_ok=True)


logging.basicConfig(
    filename=log_path,
    filemode="a",
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO
)

logging.info("===== PREPROCESSED DATA LOADER BOSHLANDI =====")


BASE_PATH = r"C:\Users\Rasulbek907\Desktop\Hotel Booking Cancellation Prediction\Data\Preprosessed"

PATHS = {
    "X_train": "X_train_preprocessed.csv",
    "X_test":  "X_test_preprocessed.csv",
    "y_train": "y_train.csv",
    "y_test":  "y_test.csv"
}


try:
    X_train = pd.read_csv(os.path.join(BASE_PATH, PATHS["X_train"]))
    X_test  = pd.read_csv(os.path.join(BASE_PATH, PATHS["X_test"]))
    y_train = pd.read_csv(os.path.join(BASE_PATH, PATHS["y_train"]))
    y_test  = pd.read_csv(os.path.join(BASE_PATH, PATHS["y_test"]))

    logging.info("Preprocessed datasetlar muvaffaqiyatli yuklandi")
    logging.info(f"X_train shape: {X_train.shape}")
    logging.info(f"X_test  shape: {X_test.shape}")
    logging.info(f"y_train shape: {y_train.shape}")
    logging.info(f"y_test  shape: {y_test.shape}")

except Exception as e:
    logging.error(f"Datasetlarni yuklashda xatolik: {e}")
    raise


if X_train.shape[0] != y_train.shape[0]:
    logging.error("X_train va y_train satr soni mos emas")
    raise ValueError("Train set mismatch")

if X_test.shape[0] != y_test.shape[0]:
    logging.error("X_test va y_test satr soni mos emas")
    raise ValueError("Test set mismatch")

# Target leakage tekshiruvi
if set(y_train.columns) & set(X_train.columns):
    logging.error("Target X_train ichiga kirib ketgan!")
    raise ValueError("Target leakage detected")

logging.info("DLP tekshiruvlar muvaffaqiyatli o‘tdi")
logging.info("===== DATA LOADER YAKUNLANDI =====")